<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Movie Reviews

In this exercise we are given a set of IMDB movie reviews and we train our model to predict other reviews. The output of the review is either positive or negative so it is a binary classification problem like the ship/not ship problem we dealt with before

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np
#import cupy as np
from keras.utils import to_categorical



## The data

THe movie review dataset is a set of 50000 reviews of movies (half training, half test). Each review contains a set of words and is labeled positive (1) or negative (0). For convenience each word index refer to its frequency of occurence in the dataset. For example a word with index 5 is the fifth most frequently used data set. The indices 0,1 and 2 are reserved so 5 really means the third most frequent.

Details about the dataset can be found here [Keras IMDB](https://keras.io/api/datasets/imdb/)

In this exercise we choose only the first 10000 most frequent words to be included. Any word that is not among them is given the index 2.

First we load data set without omitting any words

### Data details

We would like to have an idea about the number of reviews, the average length of a review. Also we compute how many entries with values 0,1,2 and 3. The number 0 is used for padding and 1 to denote the beginning of each sequence. The number 2 is used for missing words. Finally, the number 3 is never used since as you will see later we will shift the indices by 3.

In [8]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.imdb.load_data()

print("The number of reviews in the x_train data set = {}\n".format(x_train.shape[0]))
print("The average length of reviews = {}".format(np.mean([len(x) for x in x_train])))
print("With standard deviation = {}".format(np.std([len(x) for x in x_train])))
print("The number of 0's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==0])))
print("The number of 1's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==1])))
print("The number of 2's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==2])))
print("The number of 3's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==3])))

The number of reviews in the x_train data set = 25000

The average length of reviews = 238.71364
With standard deviation = 176.49367364852034
The number of 0's in the x_train data set = 0

The number of 1's in the x_train data set = 25000

The number of 2's in the x_train data set = 1

The number of 3's in the x_train data set = 0



Now when we choose only the first _max_words_ most frequent words and compute the number of 2's in the data set. As you can see the number of 2's is now very large since all the "ignored" words were given the code 2.

In [2]:
max_words=5000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_words)
print("The number of 2's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==2])))

17465344/17464789 [==============================] - 1s 0us/step
The number of 2's in the x_train data set = 592372



### Word index

Keras provides also a dictionary of word to index. We don't need it for training but it helps us get an idea what the reviews are saying in plain English. From that we build a dictionary of index to words. We use the index_to_word to display the first review in the data set.

In [10]:

word_to_index=imdb.get_word_index()
index_to_word=dict([(key,val) for (val,key) in word_to_index.items()])
review = " ".join( [index_to_word.get(i - 3, "***") for i in x_train[0]] )
print(review)







## One hot Encoding

Even though this problem is similar to the image classification problem that we saw we cannot feed the word indices to our model directly. First, not all the reviews are the same length. Second, the same word in different reviews can occur at different position. For exam the two reviews " that was a good movie" and "That movie was good" will be interpreted differently because the **same** words occur at different positions. So we need to do the following

1. Truncate or pad all reviews to contain the same number of words
1. Make sure the same word occurs at the same position in every review. This we do by using one hot encoding.

Suppose that our vocabulary contains only 3 words labeled 1,2 and 3. Further, suppose that two reviews have the values [2,1]  and [1,2] respectively then the one hot representation of both is the same:
[1,1,0] i.e. words 1 and 2 are present but 3 is missing

**NOTE** our representation is not very efficient. A better approach is to use **word embedding** which we don't use here.



## Implementation Details

The indices of a numpy tensor can be arrays. For example suppose that we are given an array A of size _n_ filled with zeros and we need to set the values at positions 1,13,27 with 1. We can perform the operation in one statement as A[[1,13,27]]=1. Knowing that the function one_hot below goes through every review which is a array of indices and sets the corresponding positions to 1.

In [4]:


def one_hot(reviews, nb_words = max_words):
 res = np.zeros((len(reviews), nb_words))
 for i, review in enumerate(reviews):
  res[i, review] = 1
 return res

x_train_one_hot=one_hot(x_train)
x_test_one_hot=one_hot(x_test)
print(x_train.shape)


(25000,)


In [6]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1,input_shape=(x_train_one_hot.shape[1],),activation="sigmoid"))
model.summary()
model.compile(optimizer="Adam",loss="binary_crossentropy",
    metrics=["accuracy"])
history = model.fit(
    x_train_one_hot,
    y_train,
    batch_size=500,
    epochs=10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 5001      
Total params: 5,001
Trainable params: 5,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
50/50 [==============================] - 0s 6ms/step - loss: 0.6086 - accuracy: 0.7283
Epoch 2/10
50/50 [==============================] - 0s 6ms/step - loss: 0.4905 - accuracy: 0.8424
Epoch 3/10
50/50 [==============================] - 0s 5ms/step - loss: 0.4276 - accuracy: 0.8641
Epoch 4/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3870 - accuracy: 0.8760
Epoch 5/10
50/50 [==============================] - 0s 5ms/step - loss: 0.3579 - accuracy: 0.8845
Epoch 6/10
50/50 [==============================] - 0s 5ms/step - loss: 0.3361 - accuracy: 0.8910
Epoch 7/10
50/50 [==============================] - 0s 5ms

## Word Embedding

A better approach is to use word embedding. First we truncate/pad all reviews to the same length using the pad_sequences function from Keras. Then we add an Embedding layer as the first layer in our model.

### EXAMPLE

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(6, 1, input_length=4))
#model.add(tf.keras.layers.Flatten())
#input_array = np.random.randint(10, size=(3, 4))
input_array=np.array([[1,2,3,4],[4,3,2,1],[3,3,4,5]])
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape)
print(output_array)

## The Model

In [23]:
x_train=pad_sequences(x_train,maxlen=500)
x_test=pad_sequences(x_test,maxlen=500)

model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words,32,input_length=500))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"])
history = model.fit(
    x_train,
    y_train,
    batch_size=500,
    epochs=20)


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
flatten_18 (Flatten)         (None, 16000)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16001     
Total params: 176,001
Trainable params: 176,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
50/50 [==============================] - 1s 20ms/step - loss: 0.6795 - accuracy: 0.5668
Epoch 2/20
50/50 [==============================] - 1s 20ms/step - loss: 0.5198 - accuracy: 0.7863
Epoch 3/20
50/50 [==============================] - 1s 20ms/step - loss: 0.3314 - accuracy: 0.8761
Epoch 4/20
50/50 [==============================] - 1s 21ms/step - loss: 0.2576 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5001      
Total params: 5,001
Trainable params: 5,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
50/50 [==============================] - 0s 5ms/step - loss: 0.6142 - accuracy: 0.7194
Epoch 2/2
50/50 [==============================] - 0s 5ms/step - loss: 0.4948 - accuracy: 0.8416


In [ ]:
x_train=x_train.T
x_test=x_test.T

In [ ]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [ ]:
def initialize_with_zeros(dim):
    
#w is a column vector. later on we take the transpose before operating on w
    w = np.zeros((1,dim))
    b = 0
    
    return w, b

In [ ]:
def propagate(w, b, X, Y):
# m is the number of samples which is the number of columns in X
    m = X.shape[1]
    Y_hat= sigmoid(np.dot(w,X)+b)   # compute activation
   
    cost=-np.sum(Y*np.log(Y_hat)+(1-Y)*np.log(1-Y_hat))/m
    # Compute Derivatives

    dw = np.dot(X,(Y_hat-Y).T)/m
    db = np.sum(Y_hat-Y)/m
    cost = np.squeeze(cost)
    
    return dw, db, cost


In [ ]:
def learn(w, b, X, Y, num_iterations,learning_rate, print_cost = False):
    
    for i in range(num_iterations):
        # Cost and gradient calculation 
        dw,db, cost = propagate(w,b,X,Y)
        
        # update rule
        w = w-learning_rate*dw.T
        b = b-learning_rate*db

        if print_cost and i % 500 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    
    return w,b,dw,db

In [ ]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    
    # Compute vector "Y_hat" predicting
    #    the probabilities of review  being positive

    Y_hat = sigmoid(np.dot(w,X)+b)

    for i in range(Y_hat.shape[1]):
        
        # Convert probabilities Y_hat[0,i] to actual predictions p[0,i]
        if Y_hat[0,i]>=0.5:
            Y_prediction[0,i]=1
        else:
            Y_prediction[0,i]=0
    
    assert(Y_prediction.shape == (1, m))

    return Y_prediction

In [ ]:
# initialize parameters with zeros 
w, b = initialize_with_zeros(x_train.shape[0])



In [ ]:
num_iterations = 2000
learning_rate = 0.2
print_cost = True

# Gradient descent 
w,b,dw,db= learn(w, b, x_train, y_train, num_iterations, learning_rate, print_cost)

# Predict test/train set examples
Y_prediction_test = predict(w, b, x_test)
Y_prediction_train = predict(w, b, x_train)

# Print train/test Errors

print("train accuracy:"+str((100 - np.mean(np.abs(Y_prediction_train - y_train)) * 100)))
print("test accuracy:"+str((100 - np.mean(np.abs(Y_prediction_test - y_test)) * 100)))

Cost after iteration 0: 0.693147
Cost after iteration 500: 0.302174
Cost after iteration 1000: 0.266001
Cost after iteration 1500: 0.247101
train accuracy:91.428
test accuracy:88.3
